In [ ]:
import sys, os, git, time

current_path = os.getcwd()
git_repo = git.Repo(current_path, search_parent_directories=True)
git_path = git_repo.git.rev_parse("--show-toplevel")
sys.path.append(git_path+"/python-codes/")

from InitializeSimulation import InitializeSimulation
from Utilities import Utilities
from Outputs import Outputs
from MolecularDynamics import MolecularDynamics
from MonteCarlo import MonteCarlo

import numpy as np

In [ ]:
self = MolecularDynamics(number_atoms=[5, 5],
                      epsilon=[0.1, 0.1], # kcal/mol
                      sigma=[2, 1], # A
                      atom_mass=[1, 1], # g/mol
                      atom_charge=[0,0], # in elementary charge units
                      Lx=30, # A
                      Ly=30, # A
                      Lz=30, # A
                      minimization_steps = 50,
                      maximum_steps=500,
                      desired_temperature=300,
                      thermo = 10,
                      dump = 10,  
                      tau_temp = 100, # fs
                      tau_press= 1000, # fs
                      time_step=1, # fs
                      seed=319817,
                      data_folder = "mdcode-output/",
                      )
self.run()
# run lammps for comparison
os.system("/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial -in input.lammps > /dev/null")